# EJERCICIO DE PANDAS
Se te ha proporcionado un conjunto de datos donde su finalizad es entrenar una IA que prediga que tan probables es una persona que padezca obesidad, los campos son los siguientes
1. Género: Género de la persona

2. Age: Edad de la persona

3. Height: Altura en **metros**

4. Weight: Peso en **kilogramos**

5. family_history: Sus familiares sufrieron o no obesidad 

6. FAVC: Consumo frecuente de calorías

7. NCP: Cuantas comidas principales se consume

8. SMOKE: La persona fuma

9. CH20: Cuanta agua bebe la persona diariamente

10. SCC: Se controlan las calorias que se consumen

11. FAF: Cuanta actividad fisica se realiza

12. CALC: Frecuencia en la que la persona afectada bebe o no alcohol

13. MTRANS: Transporte que la persona suele usar

14. Obesity_level: nivel de obesidad **ESTE CAMPO ES INTOCABLE**

Pasos a realizar:
1. Primera vista de los datos
2. Visualizacion de los datos 
3. Detección de nulos
4. Transformación de los datos (solo es necesario el campo gender y CALC)
5. Estudio de correlación
6. Escalado y prueba con el modelo

**IMPORTANTE** La parte en la que pone entrenamiento del modelo no la toqueis ya que si no no se os podrá evaluar la actividad, cualquier duda preguntad

**PISTAS**
- Para la visualizacion de datos categoricos dividelo en 3 celdas:
    <ol>
        <li>Campos: Gender,FAVC,CAEC</li>
        <li>Campos: SMOKE,SCC,CALC</li>
        <li>Campos: MTRANS</li>
    </ol>
    Esto se hace para que no se colapse la informacion

- Para la visualizacion de datos en 2D utilizar estos campos obesidad.get(["Age","Height","Weight","FCVC","NCP"]) ya que el resto son booleanos numericos que fastidian la gráfica

- A la hora de detectar los nulos si quereis imputar varios valores a la vez una vez que los hayais transformados teneis que crear un DataFrame con los valores transformados, borrar los campos transformados del DataFrame original (aun no se habrán reemplazado) y usar obesidad = pd.concat([obesidad,c_transformados])

- Para hacer la correlacion se tendrá que usar este DataFrame correlacion = obesidad.drop(columns=["family_history","FAVC","SMOKE","SCC","MTRANS"]) ya que implica parsear el campo obesity a ordinal, se colocará la transformacion en las pistas
```python
objetivo = correlacion["Obesity"]
ord_encoder = OrdinalEncoder(categories=[["Insufficient_Weight","Normal_Weight","Overweight_Level_I","Overweight_Level_II","Obesity_Type_I","Obesity_Type_II","Obesity_Type_III"]])
objetivo = objetivo.to_numpy()
objetivo = objetivo[:,np.newaxis]
ord_encoder.fit(objetivo)
correlacion["Obesity"] = ord_encoder.transform(objetivo)
correlacion.head()
```

## IMPORTACIÓN DE LIBRERIAS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [3]:
# IMPORTANTE EL RESULTADO FINAL DEBE DE SER ESTE DATAFRAME PARA EL ENTRENAMIENTO DEL MODELO
# NO LE CAMBIESIS EL NOMBRE O USEIS OTRO
obesidad = pd.read_csv("./obesity_dataset.csv")
obesidad.head()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


# TU EJERCICIO EMPIEZA AQUI

# ENTRENAMIENTO DEL MODELO, **NO TOQUEIS ESTA PARTE**

In [ ]:
# transformacion de las variables categoricas
obesidad.info()

In [ ]:
hot_encoder = OneHotEncoder()
family_h = obesidad["family_history"].to_numpy()
family_h = family_h[:,np.newaxis]
hot_encoder.fit(family_h)
family_h = hot_encoder.transform(family_h).toarray()
cat_family = pd.DataFrame(data=family_h,columns=hot_encoder.get_feature_names_out(["family_history"]))
favc = obesidad["FAVC"].to_numpy()
favc = favc[:,np.newaxis]
hot_encoder.fit(favc)
favc = hot_encoder.transform(favc).toarray()
cat_favc = pd.DataFrame(data=favc,columns=hot_encoder.get_feature_names_out(["FAVC"]))
smoke = obesidad["SMOKE"].to_numpy()
smoke = smoke[:,np.newaxis]
hot_encoder.fit(smoke)
smoke = hot_encoder.transform(smoke).toarray()
cat_smoke = pd.DataFrame(data=smoke,columns=hot_encoder.get_feature_names_out(["SMOKE"]))
scc = obesidad["SCC"].to_numpy()
scc = scc[:,np.newaxis]
hot_encoder.fit(scc)
scc = hot_encoder.transform(scc).toarray()
cat_scc = pd.DataFrame(data=scc,columns=hot_encoder.get_feature_names_out(["SCC"]))
mtrans = obesidad["MTRANS"].to_numpy()
mtrans = mtrans[:,np.newaxis]
hot_encoder.fit(mtrans)
mtrans = hot_encoder.transform(mtrans).toarray()
cat_mtrans = pd.DataFrame(data=mtrans,columns=hot_encoder.get_feature_names_out(["MTRANS"]))
obesidad = pd.concat([obesidad.drop(columns=["family_history","FAVC","SMOKE","SCC","MTRANS"]),cat_family,cat_favc,cat_smoke,cat_scc,cat_mtrans],axis=1)
obesidad.head()


In [ ]:
SPLITS = 8
MAX_NEIGHBORS = 40

np.random.seed(148)
train, test = train_test_split(obesidad,test_size=0.2,train_size=0.8)

cross_validation = KFold(n_splits=SPLITS,shuffle=True)
hiperparam = {}
for i,weights in enumerate(["uniform","distance"]):
    scores = []
    # iterado de numero de vecinos para saber cual es el numero que mejor se ajusta
    for neighbor in range(1,MAX_NEIGHBORS):
        accuracy = []
        knn = KNeighborsClassifier(neighbor,weights=weights)
        # prueba del modelo del minimo de vecinos al maximo de vecinos
        for train_fold,test_fold in cross_validation.split(train):
            
            # seleccion aleatoria de datos de entrenamiento mediante indices
            r_train = train.iloc[train_fold]
            r_test = train.iloc[test_fold]

            # entrenamiento del modelo
            knn.fit(r_train.drop(columns=["Obesity"]),r_train["Obesity"])

            # prediccion del modelo usando los datos de entrenamiento 
            evaluation = knn.predict(r_test.drop(columns=["Obesity"]))

            accuracy.append(accuracy_score(r_test["Obesity"],evaluation))

        scores.append(np.mean(accuracy))
    
    # pintado de los puntos de los pesos que mejor encajan en el algoritmo
    plt.plot(range(1, len(scores)+1), scores,
             marker='o', label=weights)
    print ('Valor maximo  ' +  weights + " : " +  str(max(scores)) + " (" + str(np.argmax(scores) + 1) + ")")
    # guardamos el valor máximo del peso para evaluarlo más adelante
    hiperparam[weights] = np.argmax(scores)+1
    plt.ylabel('Acc')

In [ ]:
np.random.seed(324)
train_x,test_x,train_y,test_y = train_test_split(obesidad.drop(columns=["Obesity"]),obesidad["Obesity"])
neighbors = 0
weight = ""
# seleccionamos el peso que mejor se ajusta y el número de vecinos recomendado
for key in hiperparam:
    if(hiperparam[key]>neighbors):
        weight = key
        neighbors = hiperparam[key]
knn = KNeighborsClassifier(n_neighbors=neighbor,weights=weight)
knn.fit(train_x,train_y)
evaluation = knn.predict(test_x)
print(f"Precision del modelo {accuracy_score(test_y,evaluation)*100}%")
print(f"INFORME DE CLASIFICACION:\n {classification_report(test_y,evaluation)}")
print("\nMATRIZ DE CONFUSION")
sb.heatmap(confusion_matrix(test_y,evaluation),annot=True,cmap="Oranges",xticklabels=obesidad["Obesity"].unique(),yticklabels=obesidad["Obesity"].unique())